In [0]:
from pyspark.sql.functions import *

# Read new data from Bronze
bronze_df = spark.table("bronze_nyc_taxi").filter("processed_date IS NULL")

# Data cleaning and transformations
silver_df = bronze_df.withColumn("trip_duration_min", 
                (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime"))/60) \
             .withColumn("is_airport_pickup", col("PULocationID").isin([1, 132, 138])) \
             .withColumn("time_of_day_slot", 
                when(hour("tpep_pickup_datetime").between(5,11), "Morning")
                .when(hour("tpep_pickup_datetime").between(12,16), "Afternoon")
                .when(hour("tpep_pickup_datetime").between(17,20), "Evening")
                .otherwise("Night")) \
             .withColumn("trip_id", sha2(concat_ws("||", 
                col("VendorID"), 
                col("tpep_pickup_datetime"), 
                col("tpep_dropoff_datetime")), 256))

# Data quality checks
assert bronze_df.filter("trip_distance <= 0").count() == 0, "Invalid trip distances found"
assert bronze_df.filter("fare_amount < 0").count() == 0, "Negative fares found"
assert bronze_df.filter("tpep_dropoff_datetime <= tpep_pickup_datetime").count() == 0, "Invalid time ranges"

# Write to Silver with merge
silver_df.createOrReplaceTempView("updates")

spark.sql("""
  MERGE INTO silver_nyc_taxi target
  USING updates source
  ON source.trip_id = target.trip_id
  WHEN MATCHED AND target.processed_date IS NULL THEN UPDATE SET *
  WHEN NOT MATCHED THEN INSERT *
""")

# Mark Bronze records as processed
spark.sql("""
  UPDATE bronze_nyc_taxi
  SET processed_date = current_date()
  WHERE processed_date IS NULL
""")